In [23]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [24]:
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.layers.experimental.preprocessing import CenterCrop
from tensorflow.keras import layers

In [45]:
# Create a dataset. 
dataset_images_training = keras.preprocessing.image_dataset_from_directory(
  '../datasets/teams/train/', 
  batch_size=4, 
  image_size=(40, 50),
  labels='inferred',
  subset="training",
  validation_split=0.2,
  seed=123)


Found 20 files belonging to 2 classes.
Using 16 files for training.


In [46]:
# Create a dataset. 
dataset_images_validation = keras.preprocessing.image_dataset_from_directory(
  '../datasets/teams/train/', 
  validation_split=0.2,
  subset="validation",
  seed=123,
  batch_size=4,
  image_size=(40, 50),
  labels='inferred')

Found 20 files belonging to 2 classes.
Using 4 files for validation.


In [48]:
class_names = dataset_images_training.class_names
print(class_names)

['blue', 'red']


In [26]:
#creating a classifier
#None means that the size of the images are Unknown
inputs = keras.layers.Input(shape=(40, 50, 3))


In [27]:
# Center-crop images to 40x40
x = CenterCrop(height=40, width=40)(inputs)





In [28]:
# Rescale images to [0, 1]
x = Rescaling(scale=1.0 / 255)(x)

In [29]:
# Apply some convolution and pooling layers
x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(3, 3))(x)
x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(3, 3))(x)
x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)

# Apply global average pooling to get flat feature vectors
x = layers.GlobalAveragePooling2D()(x)

In [30]:
# Add a dense classifier on top
num_classes = 2
outputs = layers.Dense(num_classes, activation="softmax")(x)

In [31]:
model = keras.Model(inputs=inputs, outputs=outputs)

In [32]:
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 40, 50, 3)]       0         
_________________________________________________________________
center_crop_3 (CenterCrop)   (None, 40, 40, 3)         0         
_________________________________________________________________
rescaling_2 (Rescaling)      (None, 40, 40, 3)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 38, 38, 32)        896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 10, 10, 32)        9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 3, 3, 32)          0   

In [34]:
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
              loss=keras.losses.CategoricalCrossentropy())

In [67]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [74]:
callbacks_custom = [
    keras.callbacks.ModelCheckpoint(
        filepath='saved/classification_model_epoch_{epoch}',
        save_freq='epoch'
        ),
        keras.callbacks.TensorBoard(log_dir='./logs')
]

2021-08-21 23:58:16.059254: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-08-21 23:58:16.059280: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-08-21 23:58:16.059702: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1661] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.
2021-08-21 23:58:16.059744: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-08-21 23:58:16.059763: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1752] function cupti_interface_->Finalize()failed with error CUPTI could not be loaded or symbol could not be found.


In [75]:
history=model.fit(
  dataset_images_training,
  validation_data=dataset_images_validation,
  epochs=2,
  callbacks=callbacks_custom
)

Epoch 1/2
4/4 [==============================] - 0s 19ms/step - loss: 0.0100 - accuracy: 1.0000 - val_loss: 0.0116 - val_accuracy: 1.0000


2021-08-21 23:58:25.722382: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-08-21 23:58:25.722412: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-08-21 23:58:25.722441: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1661] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.
2021-08-21 23:58:25.726281: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-08-21 23:58:25.726346: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1752] function cupti_interface_->Finalize()failed with error CUPTI could not be loaded or symbol could not be found.
2021-08-21 23:58:25.726800: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 0 callback api events and 0 activity events. 
2021-08-21 23:58:25.728536: I tensorflow

INFO:tensorflow:Assets written to: saved/classification_model_epoch_1/assets
Epoch 2/2
4/4 [==============================] - 0s 9ms/step - loss: 0.0083 - accuracy: 1.0000 - val_loss: 0.0051 - val_accuracy: 1.0000
INFO:tensorflow:Assets written to: saved/classification_model_epoch_2/assets


In [76]:
print(history.history)

{'loss': [0.009976721368730068, 0.008316749706864357], 'accuracy': [1.0, 1.0], 'val_loss': [0.011578495614230633, 0.0050577279180288315], 'val_accuracy': [1.0, 1.0]}
